# **HOW TO TRAIN NEW MODEL (En to Vi)**



Phiên bản V1 nhằm chạy thử model TED anh - việt với các đặc điểm:

1 - Chạy  7 epoch

2 - Thực hiện trên máy local


## Data Pre-processing

In [2]:
import editdistance
editdistance.eval('banana', 'bahama')

2

In [ ]:
# # Download and prepare the data
!bash clc_fairseq/examples/translation/prepare-en-vi-iwslt.sh


Cloning Moses github repository (for tokenization scripts)...
Cloning into 'mosesdecoder'...
remote: Enumerating objects: 20, done.
remote: Counting objects: 100% (20/20), done.
remote: Compressing objects: 100% (17/17), done.
remote: Total 147592 (delta 5), reused 11 (delta 3), pack-reused 147572
Receiving objects: 100% (147592/147592), 129.76 MiB | 7.84 MiB/s, done.
Resolving deltas: 100% (114030/114030), done.
Checking out files: 100% (3470/3470), done.
Cloning Subword NMT repository (for BPE pre-processing)...
Cloning into 'subword-nmt'...
remote: Enumerating objects: 576, done.
remote: Total 576 (delta 0), reused 0 (delta 0), pack-reused 576
Receiving objects: 100% (576/576), 233.12 KiB | 3.33 MiB/s, done.
Resolving deltas: 100% (349/349), done.
--2020-11-21 06:37:56--  https://github.com/hoangtrungchinh/clc_data/raw/master/en_vi_iwslt.tar.gz
Resolving github.com (github.com)... 192.30.255.113
Connecting to github.com (github.com)|192.30.255.113|:443... connected.
HTTP request sen

# Training

In [1]:
!mkdir -p "iwslt-checkpoints-v2/fconv_wmt_en_vi"
!CUDA_VISIBLE_DEVICES=0 fairseq-train "iwslt-data-bin/tokenized.en-vi" \
    --arch transformer --share-decoder-input-output-embed \
    --optimizer adam --adam-betas '(0.9, 0.98)' --clip-norm 0.0 \
    --lr 5e-4 --lr-scheduler inverse_sqrt --warmup-updates 400 \
    --dropout 0.3 --weight-decay 0.0001 \
    --criterion label_smoothed_cross_entropy --label-smoothing 0.1 \
    --max-tokens 4096 \
    --eval-bleu \
    --eval-bleu-args '{"beam": 5, "max_len_a": 1.2, "max_len_b": 10}' \
    --eval-bleu-detok moses \
    --eval-bleu-remove-bpe \
    --eval-bleu-print-samples \
    --best-checkpoint-metric bleu --maximize-best-checkpoint-metric \
    --save-dir "iwslt-checkpoints-v2/fconv_wmt_en_vi"


2020-11-25 21:38:41 | INFO | fairseq_cli.train | {'common': {'_name': None, 'no_progress_bar': False, 'log_interval': 100, 'log_format': None, 'tensorboard_logdir': None, 'wandb_project': None, 'seed': 1, 'cpu': False, 'tpu': False, 'bf16': False, 'memory_efficient_bf16': False, 'fp16': False, 'memory_efficient_fp16': False, 'fp16_no_flatten_grads': False, 'fp16_init_scale': 128, 'fp16_scale_window': None, 'fp16_scale_tolerance': 0.0, 'min_loss_scale': 0.0001, 'threshold_loss_scale': None, 'user_dir': None, 'empty_cache_freq': 0, 'all_gather_list_size': 16384, 'model_parallel_size': 1, 'quantization_config_path': None, 'profile': False}, 'common_eval': {'_name': None, 'path': None, 'post_process': None, 'quiet': False, 'model_overrides': '{}', 'results_path': None}, 'distributed_training': {'_name': None, 'distributed_world_size': 1, 'distributed_rank': 0, 'distributed_backend': 'nccl', 'distributed_init_method': None, 'distributed_port': -1, 'device_id': 0, 'local_rank': 0, 'distribut

2020-11-25 21:38:41 | INFO | fairseq_cli.train | TransformerModel(
  (encoder): TransformerEncoder(
    (dropout_module): FairseqDropout()
    (embed_tokens): Embedding(7768, 512, padding_idx=1)
    (embed_positions): SinusoidalPositionalEmbedding()
    (layers): ModuleList(
      (0): TransformerEncoderLayer(
        (self_attn): MultiheadAttention(
          (dropout_module): FairseqDropout()
          (k_proj): Linear(in_features=512, out_features=512, bias=True)
          (v_proj): Linear(in_features=512, out_features=512, bias=True)
          (q_proj): Linear(in_features=512, out_features=512, bias=True)
          (out_proj): Linear(in_features=512, out_features=512, bias=True)
        )
        (self_attn_layer_norm): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
        (dropout_module): FairseqDropout()
        (activation_dropout_module): FairseqDropout()
        (fc1): Linear(in_features=512, out_features=2048, bias=True)
        (fc2): Linear(in_features=2048, out_fe

2020-11-25 21:38:41 | INFO | fairseq.data.data_utils | loaded 109322 examples from: iwslt-data-bin/tokenized.en-vi/train.en-vi.en
2020-11-25 21:38:41 | INFO | fairseq.data.data_utils | loaded 109322 examples from: iwslt-data-bin/tokenized.en-vi/train.en-vi.vi
2020-11-25 21:38:41 | INFO | fairseq.tasks.translation | iwslt-data-bin/tokenized.en-vi train en-vi 109322 examples
epoch 001:   0%|                                        | 0/828 [00:00<?, ?it/s]2020-11-25 21:38:42 | INFO | fairseq.trainer | begin training epoch 1
epoch 001: 100%|▉| 827/828 [1:12:02<00:05,  5.22s/it, loss=7.811, nll_loss=6.9722020-11-25 22:50:49 | INFO | fairseq_cli.train | begin validation on "valid" subset

epoch 001 | valid on 'valid' subset:   0%|               | 0/46 [00:00<?, ?it/s]2020-11-25 22:51:00 | INFO | fairseq.tasks.translation | example hypothesis: đây là một phần lớn.
2020-11-25 22:51:00 | INFO | fairseq.tasks.translation | example reference: nhưng cũng có một hạn chế.

epoch 001 | valid on 'valid


epoch 001 | valid on 'valid' subset:  48%|██▊   | 22/46 [03:19<03:52,  9.67s/it]2020-11-25 22:54:18 | INFO | fairseq.tasks.translation | example hypothesis: họ có thể nói với họ, & quot; không phải là họ. & quot;
2020-11-25 22:54:18 | INFO | fairseq.tasks.translation | example reference: thực tế một con muỗi chỉ có thể di chuyển khoảng khoảng 200 yard trong toàn bộ cuộc đời của nó. chúng không di chuyển quá xa.

epoch 001 | valid on 'valid' subset:  50%|███   | 23/46 [03:28<03:39,  9.55s/it]2020-11-25 22:54:28 | INFO | fairseq.tasks.translation | example hypothesis: & lt; / href = & gt; / / / / / / / / / / / / / / / / / / / / & gt; & gt; & gt; / / / / / / / / / /
2020-11-25 22:54:28 | INFO | fairseq.tasks.translation | example reference: tên anh là simon kuznets và bản báo cáo mà anh gửi được gọi là & quot; national UNKNOWNTOKENINREF, 1929-1932 & quot;

epoch 001 | valid on 'valid' subset:  52%|███▏  | 24/46 [03:38<03:33,  9.69s/it]2020-11-25 22:54:38 | INFO | fairseq.tasks.translatio

epoch 001 | valid on 'valid' subset:  85%|█████ | 39/46 [06:36<01:28, 12.66s/it]2020-11-25 22:57:36 | INFO | fairseq.tasks.translation | example hypothesis: chúng ta biết rằng chúng ta có thể làm sao?
2020-11-25 22:57:36 | INFO | fairseq.tasks.translation | example reference: chúng ta đang xây dựng những kính thiên văn để tìm ra những đe doạ. đó là một bước đầu tiên tuyệt vời, vậy thì bước thứ hai là gì? bước thứ hai là nếu chúng ta nhìn thấy một cái nào hướng về chúng ta, chúng ta phải chặn nó. chúng ta chặn bằng cách nào?

epoch 001 | valid on 'valid' subset:  87%|█████▏| 40/46 [06:47<01:12, 12.04s/it]2020-11-25 22:57:49 | INFO | fairseq.tasks.translation | example hypothesis: và sau đó, một vài năm trước, chúng tôi đã bắt đầu làm việc với một trong những người, và những người khác nhau, và những người đàn ông, và những người đàn ông, và những người đàn ông, và những người đàn ông, và những người đàn ông, và những người đàn ông, và những người đàn ông, và những người đàn ông, và nhữn

...


epoch 006 | valid on 'valid' subset:  35%|██    | 16/46 [02:04<03:55,  7.84s/it]2020-11-26 05:32:02 | INFO | fairseq.tasks.translation | example hypothesis: đó là những gì chúng ta có thể làm trong những năm 70.
2020-11-26 05:32:02 | INFO | fairseq.tasks.translation | example reference: nó sẽ làm những người già phải lên tiếng từ những góc xa của thế giới để kêu gọi sự cân bằng thế giới.

epoch 006 | valid on 'valid' subset:  37%|██▏   | 17/46 [02:13<03:50,  7.94s/it]2020-11-26 05:32:10 | INFO | fairseq.tasks.translation | example hypothesis: và tôi nghĩ đó là một vấn đề mà chúng ta đang sống ở châu phi.
2020-11-26 05:32:10 | INFO | fairseq.tasks.translation | example reference: và tôi nghĩ đó là phần lớn nguyên nhân của sự suy thoái về pháp lý trong các chính phủ của ta bây giờ.

epoch 006 | valid on 'valid' subset:  39%|██▎   | 18/46 [02:21<03:42,  7.94s/it]2020-11-26 05:32:18 | INFO | fairseq.tasks.translation | example hypothesis: và khi tôi còn nhỏ, tôi không biết mình đang ở đâu

epoch 006 | valid on 'valid' subset:  74%|████▍ | 34/46 [04:42<01:59,  9.96s/it]2020-11-26 05:34:42 | INFO | fairseq.tasks.translation | example hypothesis: chúng ta có thể gọi nó là & # 91; & # 93; & # 93; & # 93; & # 93; & # 93; & # 93; & # 93; & # 93; & # 93; & # 93; & # 93; & # 93; & # 93; & # 93;
2020-11-26 05:34:42 | INFO | fairseq.tasks.translation | example reference: có lẽ nguyên tắc sát sườn nhất với chúng ta là nguyên tắc của linus torvalds, người tiên phong cho nguồn mở đó là ý tưởng của & quot; hãy lười như một con cáo. & quot;

epoch 006 | valid on 'valid' subset:  76%|████▌ | 35/46 [04:52<01:48,  9.89s/it]2020-11-26 05:34:53 | INFO | fairseq.tasks.translation | example hypothesis: và tôi muốn nói với các bạn rằng, chúng tôi đã làm việc với các nhà khoa học, và chúng tôi đã làm việc với các bạn, chúng tôi đã làm việc với các bạn, và chúng tôi đã làm việc với các bạn, các bạn biết đấy, chúng tôi đã làm việc cùng nhau.
2020-11-26 05:34:53 | INFO | fairseq.tasks.translation 

2020-11-26 05:37:15 | INFO | fairseq.checkpoint_utils | saved checkpoint iwslt-checkpoints-v2/fconv_wmt_en_vi/checkpoint6.pt (epoch 6 @ 4968 updates, score 5.43) (writing took 4.791045259000384 seconds)
2020-11-26 05:37:15 | INFO | fairseq_cli.train | end of epoch 6 (average epoch stats below)
2020-11-26 05:37:15 | INFO | train | epoch 006 | loss 6.325 | nll_loss 5.279 | ppl 38.82 | wps 622.7 | ups 0.17 | wpb 3570.1 | bsz 132 | num_updates 4968 | lr 0.000141876 | gnorm 1.042 | train_wall 4294 | wall 28714
epoch 007:   0%|                                        | 0/828 [00:00<?, ?it/s]2020-11-26 05:37:15 | INFO | fairseq.trainer | begin training epoch 7
epoch 007: 100%|▉| 827/828 [1:11:33<00:05,  5.03s/it, loss=6.244, nll_loss=5.1862020-11-26 06:48:55 | INFO | fairseq_cli.train | begin validation on "valid" subset

epoch 007 | valid on 'valid' subset:   0%|               | 0/46 [00:00<?, ?it/s]2020-11-26 06:49:04 | INFO | fairseq.tasks.translation | example hypothesis: nhưng nó là một p


epoch 007 | valid on 'valid' subset:  80%|████▊ | 37/46 [05:20<01:30, 10.07s/it]2020-11-26 06:54:28 | INFO | fairseq.tasks.translation | example hypothesis: một ngày nọ, một người đàn ông đang làm việc trong phòng thí nghiệm của mình, và một người đàn ông tên là jjima, một người đàn ông tên là roroma, một người đàn ông tên là người đàn ông và người đàn ông.
2020-11-26 06:54:28 | INFO | fairseq.tasks.translation | example reference: trong ted video lưu trữ từ năm 1998, paralympic vận động viên aimee mullins nói về sự nghiệp kỷ lục của cô như một vận động viên điền kinh, và về đôi chân giả tuyệt vời đã giúp cô băng qua vạch kết thúc.

epoch 007 | valid on 'valid' subset:  83%|████▉ | 38/46 [05:33<01:27, 10.92s/it]2020-11-26 06:54:39 | INFO | fairseq.tasks.translation | example hypothesis: một trong những lý do tại sao người ta không làm việc với những người phụ nữ, những người đàn ông, một người đàn ông, một người đàn ông, một người đàn ông, một người đàn ông, một người đàn ông, một ngư

epoch 008:   0%|                                        | 0/828 [00:00<?, ?it/s]2020-11-26 06:56:25 | INFO | fairseq.trainer | begin training epoch 8
epoch 008:  13%|▏| 110/828 [09:32<1:06:39,  5.57s/it, loss=6.168, nll_loss=5.1, ^C
Traceback (most recent call last):                                              
  File "/usr/local/bin/fairseq-train", line 33, in <module>
    sys.exit(load_entry_point('fairseq', 'console_scripts', 'fairseq-train')())
  File "/home/chinh/clc_fairseq/fairseq_cli/train.py", line 392, in cli_main
    distributed_utils.call_main(cfg, main)
  File "/home/chinh/clc_fairseq/fairseq/distributed_utils.py", line 334, in call_main
    main(cfg, **kwargs)
  File "/home/chinh/clc_fairseq/fairseq_cli/train.py", line 130, in main
    valid_losses, should_stop = train(cfg, trainer, task, epoch_itr)
  File "/usr/lib/python3.8/contextlib.py", line 75, in inner
    return func(*args, **kwds)
  File "/home/chinh/clc_fairseq/fairseq_cli/train.py", line 219, in train
    log_

In [3]:
!pwd

/home/chinh/clc_fairseq


In [4]:
# Evaluate
!mkdir -p iwslt-data-bin-v2/fconv_wmt_en_vi
!fairseq-generate \
    iwslt-data-bin/tokenized.en-vi/ \
    --path iwslt-checkpoints-v2/fconv_wmt_en_vi/checkpoint_best.pt \
    --beam 5 --remove-bpe | tee iwslt-data-bin-v2/fconv_wmt_en_vi/gen.out

2020-11-26 07:07:33 | INFO | fairseq_cli.generate | {'common': {'_name': None, 'no_progress_bar': False, 'log_interval': 100, 'log_format': None, 'tensorboard_logdir': None, 'wandb_project': None, 'seed': 1, 'cpu': False, 'tpu': False, 'bf16': False, 'memory_efficient_bf16': False, 'fp16': False, 'memory_efficient_fp16': False, 'fp16_no_flatten_grads': False, 'fp16_init_scale': 128, 'fp16_scale_window': None, 'fp16_scale_tolerance': 0.0, 'min_loss_scale': 0.0001, 'threshold_loss_scale': None, 'user_dir': None, 'empty_cache_freq': 0, 'all_gather_list_size': 16384, 'model_parallel_size': 1, 'quantization_config_path': None, 'profile': False}, 'common_eval': {'_name': None, 'path': 'iwslt-checkpoints-v2/fconv_wmt_en_vi/checkpoint_best.pt', 'post_process': '@@ ', 'quiet': False, 'model_overrides': '{}', 'results_path': None}, 'distributed_training': {'_name': None, 'distributed_world_size': 1, 'distributed_rank': 0, 'distributed_backend': 'nccl', 'distributed_init_method': None, 'distribut

2020-11-26 07:07:33 | INFO | fairseq.tasks.translation | [en] dictionary: 7768 types
2020-11-26 07:07:33 | INFO | fairseq.tasks.translation | [vi] dictionary: 6720 types
2020-11-26 07:07:33 | INFO | fairseq.data.data_utils | loaded 1080 examples from: iwslt-data-bin/tokenized.en-vi/test.en-vi.en
2020-11-26 07:07:33 | INFO | fairseq.data.data_utils | loaded 1080 examples from: iwslt-data-bin/tokenized.en-vi/test.en-vi.vi
2020-11-26 07:07:33 | INFO | fairseq.tasks.translation | iwslt-data-bin/tokenized.en-vi/ test en-vi 1080 examples
2020-11-26 07:07:33 | INFO | fairseq_cli.generate | loading model(s) from iwslt-checkpoints-v2/fconv_wmt_en_vi/checkpoint_best.pt
  0%|                                                     | 0/7 [00:00<?, ?it/s]S-1046	we were hired to replace this building , which burned down .
T-1046	chúng tôi được thuê để thay thế toà nhà này , nó bị cháy rụi .
H-1046	-1.9237571954727173	chúng tôi đã xây dựng các mô hình này .
D-1046	-1.9237571954727173	chúng tôi đã xây dựn

S-49	now , there &amp; apos ; s a profound irony here .
T-49	đến đây ta gặp một sự trớ trêu .
H-49	-1.1838785409927368	vâng , đây là một ví dụ tuyệt vời .
D-49	-1.1838785409927368	vâng , đây là một ví dụ tuyệt vời .
P-49	-4.2583 -0.1387 -1.2689 -0.2705 -0.9626 -2.3148 -0.0109 -2.6183 -0.0926 -0.9989 -0.0882
S-584	we know what this technology is worth to sports leagues .
T-584	công nghệ này đáng giá với các giải đấu ,
H-584	-1.5834282636642456	chúng ta có thể thực hiện điều này trong một thế giới mới .
D-584	-1.5834282636642456	chúng ta có thể thực hiện điều này trong một thế giới mới .
P-584	-0.3768 -0.6218 -2.8265 -2.0103 -4.3731 -1.0575 -2.3936 -0.5888 -1.8490 -2.7756 -1.2089 -0.3024 -2.5841 -0.6865 -0.0964
S-562	well , imagine you &amp; apos ; re the quarterback .
T-562	hình dung , bạn là cầu thủ chuyền bóng .
H-562	-1.4556974172592163	bạn có thể thấy , đây là một bức ảnh .
D-562	-1.4556974172592163	bạn có thể thấy , đây là một bức ảnh .
P-562	-1.7278 -1.5635 -0.2783 -0.8791 -2.2028

D-256	-1.6368842124938965	tôi muốn trở thành một đứa trẻ .
P-256	-0.3218 -3.1538 -3.7237 -0.3601 -1.1152 -3.1022 -0.2306 -2.5988 -0.1257
S-235	otherwise , she will be called disobedient .
T-235	nếu không , cô sẽ bị gọi là ngỗ ngược .
H-235	-1.5142520666122437	cô ấy nói , &amp; quot ; không .
D-235	-1.5142520666122437	cô ấy nói , &amp; quot ; không .
P-235	-4.1961 -1.0547 -2.9322 -0.9207 -1.0975 -0.1103 -0.0552 -2.2828 -2.2054 -0.2877
S-80	can i get a pancake , please ? &amp; quot ;
T-80	cháu muốn gọi một chiếc bánh kếp . &amp; quot ;
H-80	-1.0896772146224976	&amp; quot ; tôi có thể dùng nó không ? &amp; quot ;
D-80	-1.0896772146224976	&amp; quot ; tôi có thể dùng nó không ? &amp; quot ;
P-80	-2.1017 -0.0614 -0.0522 -1.2801 -0.8472 -0.8005 -3.7975 -3.1487 -2.2312 -0.1416 -0.6024 -0.0520 -0.0536 -0.0855
S-999	we push the pendulum back into the other direction .
T-999	chúng ta đẩy con lắc trở lại theo hướng kia .
H-999	-1.789076328277588	chúng tôi bắt đầu sử dụng các thiết bị này .
D-999	

D-734	-1.6841791868209839	đó là một sự thay đổi rất quan trọng , rất quan trọng .
P-734	-4.9822 -0.2285 -1.2972 -3.1933 -2.2080 -0.0223 -3.5078 -2.2914 -0.0858 -1.2386 -3.3031 -2.3646 -0.0850 -0.3702 -0.0848
S-711	it &amp; apos ; s an amazing multi-track movie .
T-711	đó là một bộ phim nhiều phần tuyệt vời
H-711	-1.3885926008224487	đó là một hình ảnh rất thú vị .
D-711	-1.3885926008224487	đó là một hình ảnh rất thú vị .
P-711	-2.0916 -0.1579 -0.5132 -4.7856 -0.4306 -3.0429 -2.3842 -0.0385 -0.3554 -0.0860
S-548	losing games gets you fired as a coach .
T-548	việc bại trận khiến họ bị sa thải .
H-548	-2.3174996376037598	bạn có thể thấy một người đàn ông .
D-548	-2.3174996376037598	bạn có thể thấy một người đàn ông .
P-548	-2.2129 -1.8432 -0.8380 -4.5386 -1.1610 -4.6141 -2.9785 -0.1384 -4.7446 -0.1056
S-370	thank you . that was so lovely . thank you .
T-370	cảm ơn . rất tuyệt . cảm ơn .
H-370	-0.44837644696235657	cảm ơn rất nhiều .
D-370	-0.44837644696235657	cảm ơn rất nhiều .
P-370	-1.307

...

S-903	thirty-one percent , nearly one in three teenagers , are withdrawing from classroom debate . they &amp; apos ; re failing to engage in classroom debate because they don &amp; apos ; t want to draw attention to the way that they look .
T-903	31 % , gần 1 / 3 các em thiếu niên rút lui khỏi buổi tranh luận của lớp vì không muốn bị chú ý về vẻ ngoài .
H-903	-2.3295867443084717	họ không biết về những gì xảy ra trong những năm 80 , trong những năm 70 , họ đã có một cuộc cách mạng mới để họ có thể làm việc trong cuộc sống của họ .
D-903	-2.3295867443084717	họ không biết về những gì xảy ra trong những năm 80 , trong những năm 70 , họ đã có một cuộc cách mạng mới để họ có thể làm việc trong cuộc sống của họ .
P-903	-1.8177 -3.8763 -2.7538 -3.9757 -2.6220 -4.0885 -2.3995 -0.0543 -1.5100 -2.4325 -0.4823 -3.1365 -0.7980 -3.5057 -2.9069 -1.4353 -3.0621 -0.4257 -1.8637 -1.9588 -3.9164 -2.6821 -3.9333 -3.5690 -0.0395 -4.1457 -1.1775 -3.9653 -2.7658 -0.6926 -4.5284 -1.7428 -2.3404 -3.8159 -2.404

T-968	điều đó có nghĩa là khi tôi cho các bạn thấy một toà nhà như thế này , tôi biết những gì bạn nghĩ : bạn nghĩ về &amp; quot ; quyền lực &amp; quot ; và &amp; quot ; ổn định &amp; quot ; và &amp; quot ; dân chủ &amp; quot ; .
H-968	-1.4678411483764648	và tôi nghĩ , &amp; quot ; các bạn biết đấy , có một vấn đề lớn về vấn đề này , và đó là lý do tại sao chúng ta có thể làm điều đó . &amp; quot ;
D-968	-1.4678411483764648	và tôi nghĩ , &amp; quot ; các bạn biết đấy , có một vấn đề lớn về vấn đề này , và đó là lý do tại sao chúng ta có thể làm điều đó . &amp; quot ;
P-968	-0.4372 -1.5525 -1.0995 -1.4640 -0.7212 -0.0523 -0.0578 -3.9059 -0.2092 -1.9731 -1.0193 -0.1596 -3.5777 -2.0295 -2.8342 -0.0805 -3.1874 -2.2738 -3.2446 -0.0830 -2.3405 -2.8554 -1.8065 -3.4469 -0.3603 -2.9325 -0.0507 -0.8444 -0.0082 -2.0763 -0.3355 -3.1080 -0.8205 -2.6712 -2.5923 -0.8521 -2.4840 -0.3742 -0.0760 -0.0605 -0.1232
S-708	and with that simple fact , i stand here as proof that violence isn &amp; apos ; t inh

 86%|██████████████████████████████▊     | 6/7 [01:19<00:14, 14.90s/it, wps=264] trẻ , và cô ấy là một đứa trẻ tuyệt vời . &amp; quot ;
P-699	-2.6129 -0.5469 -2.2184 -2.6194 -0.3366 -1.5589 -1.6199 -0.3285 -0.0682 -0.0570 -2.8990 -1.3754 -3.7706 -0.7434 -3.5294 -0.3742 -2.9976 -2.1935 -2.0846 -1.3468 -3.8889 -0.5583 -2.0609 -0.3271 -4.7129 -0.4927 -1.6406 -2.2009 -0.0639 -0.0498 -0.0878
S-149	i began to try different mixtures , and i was blown away by how tiny changes in dosages dramatically changed my sense of self , my sense of who i was , my thinking , my behavior towards people .
T-149	tôi bắt đầu thử pha trộn khác đi và thật kinh ngạc chỉ một thay đổi nhỏ về liều lượng cũng có thể thay đổi ghê gớm cảm nhận của tôi về bản thân , cách suy nghĩ và cách cư xử với người khác .
H-149	-1.7828006744384766	tôi bắt đầu nhận ra mình đang làm việc , và tôi nhận ra rằng tôi đang làm việc với công việc của mình , và tôi đã làm việc với công việc của mình .
D-149	-1.7828006744384766	tôi bắt đầu 

H-362	-1.9888797998428345	nếu bạn là một người phụ nữ , và một người phụ nữ , một người phụ nữ , một người phụ nữ , một người phụ nữ , một người phụ nữ , một người phụ nữ , một người phụ nữ , một người phụ nữ , một người phụ nữ , một người phụ nữ , một người phụ nữ , một người phụ nữ , một người phụ nữ , một người phụ nữ , một người phụ nữ , một người phụ nữ , một người phụ nữ , một người phụ nữ , và họ có thể giúp họ không làm gì với họ ?
D-362	-1.9888797998428345	nếu bạn là một người phụ nữ , và một người phụ nữ , một người phụ nữ , một người phụ nữ , một người phụ nữ , một người phụ nữ , một người phụ nữ , một người phụ nữ , một người phụ nữ , một người phụ nữ , một người phụ nữ , một người phụ nữ , một người phụ nữ , một người phụ nữ , một người phụ nữ , một người phụ nữ , một người phụ nữ , một người phụ nữ , một người phụ nữ , và họ có thể giúp họ không làm gì với họ ?
P-362	-2.6819 -2.1722 -3.1163 -1.0609 -2.0512 -1.7477 -0.0477 -2.4606 -2.6112 -3.8770 -1.1732 -1.5568 -0.0604 -3

P-56	-1.3390 -4.3283 -2.4597 -0.2553 -1.4063 -2.2200 -2.0670 -0.6348 -2.8012 -1.1335 -0.9888 -3.7587 -1.5769 -1.9907 -3.2245 -2.1665 -1.7467 -0.0184 -2.3159 -2.8726 -1.3836 -1.2830 -0.0199 -2.4903 -4.5624 -1.4455 -1.2878 -0.0192 -2.3541 -2.9848 -1.3138 -1.3669 -0.0220 -2.3210 -2.6092 -1.2142 -1.4025 -0.0207 -2.3641 -2.4236 -1.2414 -1.4214 -0.0241 -2.2740 -2.4390 -1.2896 -1.4650 -0.0256 -2.2177 -2.3572 -1.3114 -1.4990 -0.0218 -2.2525 -2.3596 -1.3233 -1.5623 -0.0244 -2.2210 -2.3992 -1.5014 -1.5696 -0.0264 -2.2249 -2.3673 -1.4307 -1.6437 -0.0247 -2.1851 -2.4103 -1.6210 -1.6668 -0.0270 -2.1281 -2.4970 -1.5484 -1.6854 -0.0274 -2.1673 -2.5683 -1.5634 -1.6852 -0.0269 -2.1934 -2.6378 -1.8457 -1.6709 -0.0274 -2.1450 -2.7419 -1.9163 -1.6497 -0.0286 -2.1561 -2.7811 -2.0130 -1.6199 -0.0308 -2.1515 -2.7936 -2.0959 -1.6500 -0.0290 -2.1523 -2.8490 -2.1030 -1.7292 -0.0299 -2.1998 -2.8365 -2.0948 -1.7528 -0.0317 -2.2108 -2.8939 -2.2069 -1.7350 -0.0358 -2.1525 -2.9094 -2.0348 -1.6713 -0.0379 -2.2050 -2.

Compute BLEU score


In [1]:
!grep ^H iwslt-data-bin-v2/fconv_wmt_en_vi/gen.out | cut -f3- > iwslt-data-bin-v2/fconv_wmt_en_vi/gen.out.sys
!grep ^T iwslt-data-bin-v2/fconv_wmt_en_vi/gen.out | cut -f2- > iwslt-data-bin-v2/fconv_wmt_en_vi/gen.out.ref
!fairseq-score --sys iwslt-data-bin-v2/fconv_wmt_en_vi/gen.out.sys --ref iwslt-data-bin-v2/fconv_wmt_en_vi/gen.out.ref


Namespace(ignore_case=False, order=4, ref='iwslt-data-bin-v2/fconv_wmt_en_vi/gen.out.ref', sacrebleu=False, sentence_bleu=False, sys='iwslt-data-bin-v2/fconv_wmt_en_vi/gen.out.sys')
BLEU4 = 3.30, 28.1/6.8/2.3/0.7 (BP=0.788, ratio=0.808, syslen=21455, reflen=26566)


BLEU4 = 3.3, cao hơn 1 chút so với V1 = 2.18, nhưng vẫn conf quá thấp, cần xem lại 

In [2]:
from fairseq.models.transformer import TransformerModel
model = TransformerModel.from_pretrained(
  'iwslt-checkpoints-v2/fconv_wmt_en_vi',
  checkpoint_file='checkpoint_best.pt',
  bpe='subword_nmt',
  bpe_codes = 'iwslt-tokenized.en-vi/code'
)
print(model.translate('I am going to school'))

cô ấy đang làm việc ở trường học .
